# Работа с Excel

Материалы:
* Макрушин С.В. Лекция 7: Работа с Excel
* https://docs.xlwings.org/en/stable/quickstart.html
* https://nbviewer.jupyter.org/github/pybokeh/jupyter_notebooks/blob/master/xlwings/Excel_Formatting.ipynb#search_text


## Задачи для совместного разбора

1. На листе "Рецептура" файла `себестоимостьА_в1.xlsx` для области "Пшеничный хлеб" рассчитать себестоимость всех видов продукции.

2. Результаты расчетов 1.1 сохранить в отдельном столбце области "Пшеничный хлеб"

3. Приблизить форматирование столбца, добавленного в задаче 2 к оформлению всей области.

4. Выполнить 3 с помощью "протягиваемых" формул.

## Лабораторная работа 7.1

In [2]:
import csv
import pandas as pd

1. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample.csv` (__ЛР5__) в виде `pd.DataFrame`. Обратите внимание на корректное считывание столбца(ов) с индексами. Оставьте в таблице с рецептами следующие столбцы: `id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`

In [3]:

reviews = pd.read_csv('reviews_sample.csv', index_col=0)
recipes = pd.read_csv('recipes_sample.csv', index_col=0, usecols=['id', 'name', 'minutes', 'submitted', 'description', 'n_ingredients'])

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,NaN,an original recipe created by chef scott meska...,18.0
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,NaN,my children and their friends ask for my homem...,NaN
2,i can t believe it s spinach,38798,30,1533,2002-08-29,NaN,"these were so go, it surprised even me.",8.0
3,italian gut busters,35173,45,22724,2002-07-27,NaN,my sister-in-law made these for us at a family...,NaN
4,love is in the air beef fondue sauces,84797,25,4470,2004-02-23,4.0,i think a fondue is a very romantic casual din...,NaN


2. Случайным образом выберите 5% строк из каждой таблицы и сохраните две таблицы на разные листы в один файл `recipes.xlsx`. Дайте листам названия "Рецепты" и "Отзывы", соответствующие содержанию таблиц. 

In [ ]:
reviews_sample = reviews.sample(frac=0.05, random_state=42)
recipes_sample = recipes.sample(frac=0.05, random_state=42)
with pd.ExcelWriter('recipes.xlsx') as writer:
    recipes_sample.to_excel(writer, sheet_name='Рецепты')
    reviews_sample.to_excel(writer, sheet_name='Отзывы')

3. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_assign`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи присваивания массива значений диапазону ячеек.

In [ ]:
recipes_wb = xw.Book('recipes.xlsx')
recipes_sheet = recipes_wb.sheets['Рецепты']
minutes_range = recipes_sheet.range('D2:D' + str(len(recipes_sample) + 1))
seconds_assign = [int(minutes * 60) for minutes in minutes_range.value]
recipes_sheet.range('G2').value = 'seconds_assign'
recipes_sheet.range('G3:G' + str(len(recipes_sample) + 1)).value = seconds_assign

4. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_formula`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи формул Excel.

In [ ]:
ecipes_sheet.range('H2').value = 'seconds_formula'
recipes_sheet.range('H3:H' + str(len(recipes_sample) + 1)).formula = '=D3*60'

5. Сделайте названия всех добавленных столбцов полужирными и выровняйте по центру ячейки.

In [ ]:
for col in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']:
    recipes_sheet.range(col + '1').api.Font.Bold = True
    recipes_sheet.range(col + '1').api.HorizontalAlignment = xw.constants.HAlign.Center
recipes_wb.save()

6. Раскрасьте ячейки столбца `minutes` в соответствии со следующим правилом: если рецепт выполняется быстрее 5 минут, то цвет - зеленый; от 5 до 10 минут - жёлтый; и больше 10 - красный.

In [ ]:
for i in range(2, len(recipessample)+2):
    minutes = recipessheet.range('D' + str(i)).value
    if minutes < 5:
        recipessheet.range('D' + str(i)).color = (0, 255, 0)
    elif minutes >= 5 and minutes <= 10:
        recipessheet.range('D' + str(i)).color = (255, 255, 0)
    else:
        recipessheet.range('D' + str(i)).color = (255, 0, 0)

7. Добавьте на лист `Рецепты`  столбец `n_reviews`, содержащий кол-во отзывов для этого рецепта. Выполните задание при помощи формул Excel.

In [ ]:
reviewscountformula = '=COUNTIFS(Отзывы!$C:$C,Рецепты!A3)'
recipessheet.range('I2').value = 'nreviews'
recipessheet.range('I3:I' + str(len(recipessample) + 1)).formula = reviewscountformula

## Лабораторная работа 7.2

8. Напишите функцию `validate()`, которая проверяет соответствие всех строк из листа `Отзывы` следующим правилам:
    * Рейтинг - это число от 0 до 5 включительно
    * Соответствующий рецепт имеется на листе `Рецепты`
    
В случае несоответствия этим правилам, выделите строку красным цветом

In [ ]:
def validate():
    reviewssheet = recipeswb.sheets['Отзывы']
    recipessheet = recipeswb.sheets['Рецепты']
    maxrow = len(reviewssheet.range('A:A').value)
    for i in range(2, maxrow+1):
        rating = reviewssheet.range('D'+str(i)).value
        recipeid = reviewssheet.range('C'+str(i)).value
        if rating is not None and (rating < 0 or rating > 5):
            reviewssheet.range('A'+str(i)+':E'+str(i)).color = (255, 0, 0)
        elif recipeid not in recipessheet.range('A2:A'+str(len(recipessample)+1)).value:
            reviewssheet.range('A'+str(i)+':E'+str(i)).color = (255, 0, 0)
validate()

9. В файле `recipes_model.csv` находится модель данных предметной области "рецепты". При помощи пакета `csv` считайте эти данные. При помощи пакета `xlwings` запишите данные на лист `Модель` книги `recipes_model.xlsx`, начиная с ячейки `A2`, не используя циклы. Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука. 

In [ ]:
import csv
import xlwings as xw


recipesmodel = pd.readcsv('recipesmodel.csv')
recipesmodelwb = xw.Book()
recipesmodelsheet = recipesmodelwb.sheets[0]
recipesmodelsheet.range('A2').value = recipesmodel

10. При помощи пакета `xlwings` добавьте в столбец J формулу для описания столбца на языке SQL. Формула должна реализовывать следующую логику:

    1\. в начале строки идут значения из столбцов В и C (значение столбца С приведено к верхнему регистру), разделенные пробелом
    
    2\. далее идут слова на основе столбца "Ключ"
        2.1 если в столбце "Ключ" указано значение "PK", то дальше через пробел идет ключевое слово "PRIMARY KEY"
        2.2 если в столбце "Ключ" указано значение "FK", то дальше через пробел идет ключевое слово "REFERENCES", затем значения столбцов H и I в формате "название_таблицы(название_столбца)"
        
    3\. если в столбце "Обязательно к заполнению" указано значение "Y" и в столбце "Ключ" указано не "PK", то дальше через пробел идет ключевое слово "NOT NULL".

Заполните этой формулой необходимое количество строк, используя "протягивание". Количество строк для протягивания определите на основе данных.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

In [ ]:
import xlwings as xw

wb = xw.Book('book.xlsx')
sheet = wb.sheets.active
last_row = sheet.range('A1').end('down').row
for row in range(2, last_row+1):
    b_value = sheet.range(f'B{row}').value
    c_value = sheet.range(f'C{row}').value.upper()
    key_value = sheet.range(f'K{row}').value
    not_null = ''
    reference = ''
    if key_value == 'PK':
        keyword = 'PRIMARY KEY'
    elif key_value == 'FK':
        keyword = 'REFERENCES'
        h_value = sheet.range(f'H{row}').value
        i_value = sheet.range(f'I{row}').value
        reference = f'{h_value}({i_value})'
    else:
        keyword = ''
    if sheet.range(f'G{row}').value == 'Y' and key_value != 'PK':
        not_null = 'NOT NULL'
    sheet.range(f'J{row}').formula = f'="{b_value} {c_value} {keyword} {reference} {not_null}"'
wb.save() 

11. При помощи пакета `xlwings` измените стилизацию листа `Модель`.
* для заголовков добавьте заливку цвета `00ccff`
* примените автоподбор ширины столбца;
* сделайте шрифт заголовков полужирным;
* добавьте таблице автофильтр.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

In [ ]:
import xlwings as xw


wb = xw.Book('book.xlsx')
sht = wb.sheets['Модель']
sht.range('A1:G1').color = (0, 204, 255)
sht.range('A1:G1').api.Font.Bold = True
sht.range('A1:G1').api.EntireColumn.AutoFit()
sht.range('A1').api.AutoFilter()
wb.save()

12. Посчитайте количество атрибутов для каждой из сущностей. Создайте лист `Статистика` и запишите в него результат группировки, начиная с ячейки "А1". Визуализируйте полученный результат при помощи столбчатой диаграммы. Сохраните полученную визуализацию на лист `Статистика`, начиная с ячейки "E2".  Сделайте скриншот листа `Статистика` и прикрепите в ячейку ноутбука.

* Вы можете воспользоваться методами для визуализации, которые поставляются вместе с объектами `pandas` (см. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot) 